In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
torch.cuda.is_available()
torch.cuda.set_device(0)

In [ ]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 128
cifar10_args.predict_batch_size = 128
cifar10_args.device = ['cuda:0']

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.01
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'AdamW'
cifar10_args.scheduler = 'OneCycle'
cifar10_args.attack = {'fgsm':(0.005,), 'pgd':(0.005,0.1,20),'deepfool':(0.005,0.1,20),'apgd-ce':(0.005,)}

activation = 'relu'
input_channel = 3
knpp = [40,80,120,160,240,320,400,480,560]
groups=40

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[6], knpp[7], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[7], knpp[8], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('adptavgpool', (1,1)), 
    ('linear', (knpp[-1], 100, (1,2,3)))
]

In [ ]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()